In [6]:
import helper
import json
import pandas as pd

with open('/Users/oysteinbruce/Documents/GitHub/SINTEF/other/ReinforcementLearning/transport/data/sammenligne_fuel_modell.json') as json_file:
    plan = json.load(json_file)

In [7]:
X_feat = ['Quantity', 'AltitudeGain', 'AltitudeLoss', 'AltitudeDeltaEndStart', 'SpeedMean', 'SumRotation']

## 44, 27, 24, 22, 47
edges_used = []
for edge in plan["transporterPlans"]['MAN 560 ( YK 23597 )']["actions"][0]['shape']['route']['edgeSchedules']:
    edges_used.append(edge['edgeId'])

length = plan["transporterPlans"]['MAN 560 ( YK 23597 )']["actions"][0]['shape']['route']['length']
duration = plan["transporterPlans"]['MAN 560 ( YK 23597 )']["actions"][0]['shape']['route']['duration']
duration_sec = helper.get_second_from_time_formatted(duration)

trip = pd.DataFrame()
for _dict in plan['spatialGraphContainer']['network']['edges']:
    if _dict['id'] in edges_used:
        _df = pd.DataFrame(_dict['coordinates'])
        trip = pd.concat([trip, _df], axis = 0)
        
speed = length/duration_sec


trip = trip.set_axis(['Longitude', 'Latitude', 'Altitude'], axis = 1, inplace = False)
trip = helper.get_informative_coords_df(trip, is_route=True)
trip['DistanceCumsum'] = trip['distance_cumulative']
trip['DistanceCumsumDiff'] = trip['DistanceCumsum'].diff()

weight = plan["transporterPlans"]['MAN 560 ( YK 23597 )']["actions"][0]['weight']
trip['Quantity'] = weight

# pred_fuel_route = helper.estimate_fuel_consumption_of_a_route(
#                 fully_route=trip,
#                 XGBmodel_name=helper.PATH_QUICK_RUN +
#                 'models_and_results10/models/without_2021-10-18.pkl',
#                 X_feat=X_feat,
#             )

In [8]:
import numpy as np
t = [0]
distances = np.sqrt(trip['x_diff']**2 + trip['y_diff']**2)
for idx, distance in enumerate(distances[1:]):
    t.append(t[idx] + distance/speed)


In [9]:
trip.index = t
trip

,Longitude,Latitude,Altitude,x,y,x_diff,y_diff,distance_diff,distance_cumulative,DistanceCumsum,DistanceCumsumDiff,Quantity
0.000000,10.377258,59.952030,254.944664,0,817,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3.271122,10.377520,59.951780,246.573839,14,789,14.0,-28.0,31.304952,31.304952,31.304952,NaN,0.0
4.300251,10.377594,59.951698,246.250767,18,780,4.0,-9.0,9.848858,41.153809,41.153809,9.848858,0.0
5.329380,10.377669,59.951616,245.927695,22,771,4.0,-9.0,9.848858,51.002667,51.002667,9.848858,0.0
6.405192,10.377744,59.951535,245.604623,27,762,5.0,-9.0,10.295630,61.298297,61.298297,10.295630,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
280.631695,10.398143,59.945376,213.444485,1167,76,-9.0,4.0,9.848858,2685.671924,2685.671924,9.848858,0.0
281.722625,10.397974,59.945406,213.939078,1157,79,-10.0,3.0,10.440307,2696.112231,2696.112231,10.440307,0.0
282.713925,10.397804,59.945435,214.465978,1148,82,-9.0,3.0,9.486833,2705.599064,2705.599064,9.486833,0.0
283.804855,10.397632,59.945461,215.631185,1138,85,-10.0,3.0,10.440307,2716.039370,2716.039370,10.440307,0.0


In [11]:

def find_course(coords):

    x = coords['x_diff'].values
    y = coords['y_diff'].values
    vals = []
    for _x, _y in zip(x[1:], y[1:]):


        if _x > 0:
            if _y > 0:
                val = np.arctan(_y/_x) * 180/ np.pi
                
            else:
                val = 270 + 90 + np.arctan(_y/_x) * 180/ np.pi

        else:
            if _x == 0:
                if _y < 0:
                add = 90
            else:
                add = np.arctan(_y/_x)


            if _y > 0:
                val = 90 + 90 + add * 180/ np.pi
            else:
                val = 180 + add * 180/ np.pi
        
        vals.append(val)


    
    return vals



vals = find_course(trip[['x_diff', 'y_diff']])
vals.insert(0, vals[0])



<ipython-input-11-9a692c7bc9e6>:18: RuntimeWarning: invalid value encountered in double_scalars
  val = 180 + np.arctan(_y/_x) * 180/ np.pi


In [13]:
trip['Course'] = vals

In [19]:
df = pd.read_pickle('/Users/oysteinbruce/Documents/GitHub/SINTEF/fuel_model/code/lol.pkl')
df['x_diff'] = df['x'].diff()
df['y_diff'] = df['y'].diff()

In [41]:
from geopy.distance import bearing
# coordinates for two locations
coord1 = (45.5017, -73.5673)
coord2 = (43.6532, -79.3832)

# calculate the course between the two coordinates
course = bearing(coord1, coord2)

print(course)  # prints the course in degrees

ImportError: cannot import name 'bearing' from 'geopy.distance' (/opt/anaconda3/lib/python3.8/site-packages/geopy/distance.py)

In [55]:
import pyproj


def find_course1(coord1, coord2):
    geod = pyproj.Geod(ellps='WGS84')


    # coordinates for two locations

    # calculate the course between the two coordinates
    course = geod.inv(coord1[1], coord1[0], coord2[1], coord2[0])[0]

    if course < 0:
        course += 360

    return course


In [64]:
df.columns

Index(['FuelCumsum', 'data_Fuel', 'Longitude', 'Latitude', 'Altitude',
       'DistanceCumsum', 'Speed', 'x', 'y', 'Course', 'TripLogId', 'Type',
       'DumperId', 'DumperMachineName', 'DumperMachineTypeId', 'LoaderId',
       'LoaderMachineName', 'ProjectCompanyId', 'ProjectId', 'ProjectNumber',
       'ProjectExternalNumber', 'ProjectName', 'TaskId', 'TaskDescription',
       'HorizontalAccuracy', 'VerticalAccuracy', 'LoadDateTime',
       'DumpDateTime', 'MassTypeId', 'MassTypeName', 'MassTypeMaterial',
       'Quantity', 'LoadGeoFenceId', 'DumpGeoFenceId', 'LoadLatitude',
       'LoadLongitude', 'LoadHorizontalAccuracy', 'DumpLatitude',
       'DumpLongitude', 'DumpHorizontalAccuracy', 'LoadGeoFenceLongitudes',
       'LoadGeoFenceLatitudes', 'DumpGeoFenceLongitudes',
       'DumpGeoFenceLatitudes', 'load_x', 'load_y', 'dump_x', 'dump_y',
       'Course_ff', 'subtrack', 'DistanceCumsumDiff', 'FuelCumsumDiff',
       'Timestamp', 'TimeDelta', 'Forward', 'data_Fuel_v2', 'x_diff',
  

In [57]:
lon = df['Longitude']
lat = df['Latitude']

In [58]:
forward_shift_lat = lon.shift(-1)
forward_shift_lat = lat.shift(-1)

In [61]:
course_list = []

for i in range(100):
    coord1 = [lat[i], lon[i]]
    coord2 = [lat[i+1], lon[i+1]]

    course_list.append(find_course1(coord1, coord2))
    

In [63]:
for i in range(20):

    print(f"{df['Course_ff'][i+1] - df['Course_ff'][i]:7.3f} {course_list[i+1] - course_list[i]:7.3f}")

  0.000  -0.845
 -8.812  -8.365
 -3.385  -8.714
 -2.521   6.524
 -8.033  -4.530
 -0.609  -9.510
 -5.745   2.624
  2.007  -0.264
 -1.921   3.521
 -1.701  -0.012
 -0.040   4.404
  6.691   6.163
  0.000   1.569
  3.665  17.345
  0.000  -0.780
  4.336  -5.806
  3.823   7.411
  3.663  10.390
  0.903  -7.968
  0.000  -0.381


In [14]:
trip

,Longitude,Latitude,Altitude,x,y,x_diff,y_diff,distance_diff,distance_cumulative,DistanceCumsum,DistanceCumsumDiff,Quantity,Course
0.000000,10.377258,59.952030,254.944664,0,817,NaN,NaN,NaN,NaN,NaN,NaN,0.0,296.565051
3.271122,10.377520,59.951780,246.573839,14,789,14.0,-28.0,31.304952,31.304952,31.304952,NaN,0.0,296.565051
4.300251,10.377594,59.951698,246.250767,18,780,4.0,-9.0,9.848858,41.153809,41.153809,9.848858,0.0,293.962489
5.329380,10.377669,59.951616,245.927695,22,771,4.0,-9.0,9.848858,51.002667,51.002667,9.848858,0.0,293.962489
6.405192,10.377744,59.951535,245.604623,27,762,5.0,-9.0,10.295630,61.298297,61.298297,10.295630,0.0,299.054604
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280.631695,10.398143,59.945376,213.444485,1167,76,-9.0,4.0,9.848858,2685.671924,2685.671924,9.848858,0.0,156.037511
281.722625,10.397974,59.945406,213.939078,1157,79,-10.0,3.0,10.440307,2696.112231,2696.112231,10.440307,0.0,163.300756
282.713925,10.397804,59.945435,214.465978,1148,82,-9.0,3.0,9.486833,2705.599064,2705.599064,9.486833,0.0,161.565051
283.804855,10.397632,59.945461,215.631185,1138,85,-10.0,3.0,10.440307,2716.039370,2716.039370,10.440307,0.0,163.300756


In [48]:
import pickle
import os
def estimate_fuel_consumption_of_a_route(fully_route, XGBmodel_name, X_feat, group_size=30, altitude_threshold=0.01):
    """
    Sample trip pieces from df.
    :param fully_route: Formatted dataframe with Ditio and Fuel data for one route
    :param XGBmodel_name: path to the file including the XGG model
    # TODO: can make a json file to include this information, given a xgb_model
    :param X_feat: the features the model is using
    :param group_size: the amount of datapoints that should create one statistics point and be sent into the model
    :altitude_threshold:The granularity in height difference in meters per bin used to define uphill/downhill movement

    :return: the fuel prediction of the given route
    """

    fully_route['part'] = fully_route.index // group_size

    parts_of_route = [fully_route[fully_route['part'] == i]
                      for i in range(0, int(fully_route.index.values[-1])//group_size + 1)]

    # TODO: maybe do something if one part is 1 second or something
    # length_last_part = len(parts_of_route[-1])

    # compute statistics and predict fuel values for the route
    statdata = pd.DataFrame()
    for i in range(len(parts_of_route)):
        trip = parts_of_route[i]

        statdata_row = pd.Series(dtype='object')
        statdata_row['LengthDistance'] = trip['DistanceCumsum'].max(
        ) - trip['DistanceCumsum'].min()
        statdata_row['Quantity'] = trip.iloc[0].Quantity
        # Assign the type that is most frequent

        trip['AltitudeDiff'] = trip['Altitude'].diff().fillna(0)
        index_uphill = trip['AltitudeDiff'] > altitude_threshold
        index_downhill = trip['AltitudeDiff'] < -altitude_threshold
        statdata_row['AltitudeLoss'] = trip.loc[index_downhill,
                                                'AltitudeDiff'].sum()
        statdata_row['AltitudeGain'] = trip.loc[index_uphill,
                                                'AltitudeDiff'].sum()

        statdata_row['AltitudeDeltaEndStart'] = trip['Altitude'].iloc[-1] - \
            trip['Altitude'].iloc[0]

        statdata_row['AltitudeChange'] = statdata_row['AltitudeGain'] + \
            abs(statdata_row['AltitudeLoss'])

        statdata_row['AltitudeDeltaMaxMin'] = trip['Altitude'].max() - \
            trip['Altitude'].min()

        # TODO: this will be a litt bit wrong with same model?? - less time
        statdata_row['SpeedMean'] = statdata_row['LengthDistance'] / \
            len(trip)

        # Average inclinations
        average_downInclination_part = [0]
        average_upInclination_part = [0]

        for part_i in range(int(len(trip)/2)):
            part_incline = (
                trip['AltitudeDiff']/trip['DistanceCumsumDiff'])[part_i: (15 + part_i)]
            average_downInclination_part.append(
                part_incline[part_incline < 0].mean())
            average_upInclination_part.append(
                part_incline[part_incline > 0].mean())

        statdata_row['DownInclination'] = 0
        statdata_row['DownInclinationPart'] = 0
        if np.sum(index_downhill) > 0:
            statdata_row['DownInclination'] = statdata_row['AltitudeLoss'] / \
                trip.loc[index_downhill, 'DistanceCumsumDiff'].sum()
            statdata_row['DownInclinationPart'] = min(
                average_downInclination_part)

        statdata_row['UpInclination'] = 0
        statdata_row['UpInclinationPart'] = 0
        if np.sum(index_uphill) > 0:
            statdata_row['UpInclination'] = statdata_row['AltitudeGain'] / \
                trip.loc[index_uphill, 'DistanceCumsumDiff'].sum()
            statdata_row['UpInclinationPart'] = max(average_upInclination_part)

        trip['DiffCourse'] = trip.Course.diff().abs()
        statdata_row['SumRotation'] = trip['DiffCourse'][trip['DiffCourse'] < 200].sum()

        statdata_row['Sum_RotationXAltitudeDiffPos'] = (trip['DiffCourse'][(trip['DiffCourse'] < 200) & (index_uphill)]
                                                        * trip['AltitudeDiff'][(trip['DiffCourse'] < 200) & (index_uphill)]).sum()
        statdata_row['Sum_RotationXAltitudeDiffNeg'] = (trip['DiffCourse'][(trip['DiffCourse'] < 200) & (index_downhill)]
                                                        * trip['AltitudeDiff'][(trip['DiffCourse'] < 200) & (index_downhill)]).sum()

        altitudeLossPart = [0]
        altitudeGainPart = [0]
        for part_i in range(int(len(trip)/2)):
            part_altitude_change = trip['AltitudeDiff'][part_i: (15 + part_i)]
            altitudeLossPart.append(
                part_altitude_change[part_altitude_change < 0].sum())
            altitudeGainPart.append(
                part_altitude_change[part_altitude_change > 0].sum())

        statdata_row['AltitudeLossPart'] = min(altitudeLossPart)
        statdata_row['AltitudeGainPart'] = max(altitudeGainPart)

        statdata_row = statdata_row[X_feat]

        statdata = statdata.append(statdata_row, ignore_index=True)

    try:
        model_xgb_loaded = pickle.load(open(XGBmodel_name,  'rb'))
    except Exception as e:
        print(f"#### ERROR1 #### {e}")
        try:
            print(os.getcwd() + '/' + XGBmodel_name)
            model_xgb_loaded = pickle.load(
                open(os.getcwd() + '/' + XGBmodel_name,  'rb'))
        except Exception as e:
            print(f"#### ERROR2 #### {e}")
            exit()

    predicted_fuel_parts = model_xgb_loaded.predict(statdata.values)

    # scale last prediction value according to its length (in sec) - if it isn't perfectly divided
    if (len(fully_route) % group_size) != 0:
        predicted_fuel_parts[-1] = predicted_fuel_parts[-1] * \
            ((len(fully_route) % group_size)/group_size)

    predicted_fuel_fully_route = sum(predicted_fuel_parts)

    return predicted_fuel_fully_route, parts_of_route, predicted_fuel_parts



In [49]:
import helper
XGB_path = '/Users/oysteinbruce/Documents/GitHub/SINTEF/fuel_model/models/XGB/'

pred_fuel_route, parts_of_route, predicted_fuel_parts = estimate_fuel_consumption_of_a_route(
                fully_route=trip,
                XGBmodel_name=XGB_path +
                'overlap_data_2109_early_stop_on_val/without_13.pkl',
                X_feat=X_feat,
            )




<ipython-input-48-c3c7d9a1bd12>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip['AltitudeDiff'] = trip['Altitude'].diff().fillna(0)
<ipython-input-48-c3c7d9a1bd12>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip['DiffCourse'] = trip.Course.diff().abs()
<ipython-input-48-c3c7d9a1bd12>:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  statdata = statdata.append(statdata_row, ignore_index=True)
<ipython-inpu

In [58]:
print(pred_fuel_route)
predicted_fuel_parts

2.302276663482189


array([0.17686625, 0.10931554, 0.16662772, 0.18445463, 0.18309687,
       0.19215654, 0.16441077, 0.18309687, 0.5394272 , 0.40282425],
      dtype=float32)

In [59]:
parts_of_route[-2]

,Longitude,Latitude,Altitude,x,y,x_diff,y_diff,distance_diff,distance_cumulative,DistanceCumsum,DistanceCumsumDiff,Quantity,Course,part,AltitudeDiff,DiffCourse
240.403121,10.400907,59.945957,186.876063,1321,140,6.0,-8.0,10.000000,2300.680661,2300.680661,10.000000,0.0,306.869898,8.0,0.000000,NaN
241.448043,10.401012,59.945884,186.415750,1327,132,6.0,-8.0,10.000000,2310.680661,2310.680661,10.000000,0.0,306.869898,8.0,-0.460313,0.000000
242.492965,10.401117,59.945811,185.955437,1333,124,6.0,-8.0,10.000000,2320.680661,2320.680661,10.000000,0.0,306.869898,8.0,-0.460313,0.000000
243.537887,10.401222,59.945738,185.495124,1339,116,6.0,-8.0,10.000000,2330.680661,2330.680661,10.000000,0.0,306.869898,8.0,-0.460313,0.000000
244.582808,10.401327,59.945665,185.034811,1345,108,6.0,-8.0,10.000000,2340.680661,2340.680661,10.000000,0.0,306.869898,8.0,-0.460313,0.000000
245.627730,10.401432,59.945592,184.574498,1351,100,6.0,-8.0,10.000000,2350.680661,2350.680661,10.000000,0.0,306.869898,8.0,-0.460313,0.000000
246.703543,10.401532,59.945518,184.070386,1356,91,5.0,-9.0,10.295630,2360.976291,2360.976291,10.295630,0.0,299.054604,8.0,-0.504112,7.815294
247.748464,10.401627,59.945441,183.516994,1362,83,6.0,-8.0,10.000000,2370.976291,2370.976291,10.000000,0.0,306.869898,8.0,-0.553392,7.815294
248.824277,10.401721,59.945365,182.963601,1367,74,5.0,-9.0,10.295630,2381.271921,2381.271921,10.295630,0.0,299.054604,8.0,-0.553392,7.815294
252.657873,10.401916,59.945046,176.882631,1378,39,11.0,-35.0,36.687873,2417.959794,2417.959794,36.687873,0.0,287.447188,8.0,-6.080971,11.607416


In [60]:
225//30

7

In [27]:
trip['part'] = trip.index // 30
print(trip['part'].values[-1])
trip.index.values[-1]


9.0


287.63702659953134

In [28]:
parts_of_route = [trip[trip['part'] == i]
                      for i in range(0, int(trip.index.values[-1])//30)]